In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import plot
from numpy import pi as π
import firedrake
from firedrake import (inner, as_vector, max_value, min_value, sqrt, replace,
                       LinearVariationalProblem, LinearVariationalSolver,
                       grad, div, dx, ds, dS)

# Hyperbolic problems

In this demo, I'll show how to solve the conservative advection equation using the discontinuous Galerkin method.
The conservation form of this PDE is

$$\frac{d}{dt}\int_Vq\hspace{2pt}dx + \int_{\partial V}qu\cdot n\hspace{2pt}ds = \int_Vfdx$$

for any control volume $V$, where $q$ is the solution, $u$ is a velocity field, and $f$ are the sources and sinks.
This can be converted to a weak form using the method I showed earlier.
In order to use DG methods, we'll need to also add in integrals over interior facets.
As input data, we'll use the same domain as before, and we'll use the velocity field computed in the Stokes example.

In [ ]:
mesh = firedrake.Mesh('domain.msh')

As we saw with the pressure field in the last example, creating a DG function space amounts to specifying a different family. 

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='DG', degree=1)

To load the velocity field, we create another checkpoint object, only this time it's in read rather than create mode.
We then call load and pass in the name of the field we used when we stored it.

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
chk = firedrake.DumbCheckpoint('velocity', mode=firedrake.FILE_READ)
u = firedrake.Function(V)
chk.load(u, name='u')

The initial data will be equal to 1 within a circle of radius 3/8 around the point $(1/4, 1/2)$ and 0 outside.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

x0 = as_vector((1/4, 1/2))
r = 3/8
dist2 = inner(x - x0, x - x0)
expr = max_value(1 - dist2/r**2, 0)
q0 = firedrake.interpolate(expr, Q)
q = q0.copy(deepcopy=True)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tripcolor(q, cmap='viridis')
fig.colorbar(contours)
fig.show()

First, we have to decide how long to integrate the PDE for and how many timesteps to use.
We'll use an ending time of $2\pi$, which would give a signal on the boundary of one of the circles time to go all the way around.

In [ ]:
T = 2 * π
num_steps = 1200
δt = firedrake.Constant(T / num_steps)

The total flux is divided into three parts.
We can get the flux interior to each cell through the usual method or through integration by parts from the strong form of the PDE.

In [ ]:
ϕ = firedrake.TestFunction(Q)
cell_flux = -inner(grad(ϕ), q * u) * dx

Next is the flux into and out of the boundary of the domain.

In [ ]:
n = firedrake.FacetNormal(mesh)

q_in = firedrake.Constant(0)
flux_in = min_value(inner(u, n), 0) * q_in * ϕ * ds
flux_out = max_value(inner(u, n), 0) * q * ϕ * ds

Next is the flux across cell faces and this is where the DG magic comes in.
To calculate the face flux, we need to decide on a *numerical flux function*.
There are several necessary criteria for a numerical flux function to be consistent with the underlying equation, but beyond that the choice is arbitrary.
A very common one is the upwind flux, which we'll use here.

In [ ]:
u_n = 0.5 * (inner(u, n) + abs(inner(u, n)))

The face flux is expressed in terms of the jumps in the test function and the flux across the face.
We can extract the value of these functions across a face by using the call operator and passing in the characters `'+'` and `'-'` to indicate the values within the current cell and in the opposite cell.
The precise orientation is arbitrary and the resulting expression should be symmetric to interchanging the signs.
The measure that indicates integration over all cell faces, rather than just boundary faces, is `dS`.

In [ ]:
f = q * u_n
face_flux = (f('+') - f('-')) * (ϕ('+') - ϕ('-')) * dS

Finally, we can add all these together to get the total flux.

In [ ]:
F = -δt * (cell_flux + face_flux + flux_in + flux_out)

Since we'll be using an explicit time integration method, the matrix defining the linear system is just the mass matrix.

In [ ]:
ψ = firedrake.TrialFunction(Q)
m = ϕ * ψ * dx

### Solution method

To solve this PDE, we'll use the strong stability-preserving Runge Kutta method of order 3 (SSPRK3).
In each step we'll solve a linear system for the increment $\delta q$ to the solution.
SSPRK3 is a multi-stage method so we need to create some functions to store the values at each stage as well as for the total increment.

In [ ]:
δq = firedrake.Function(Q)
q1 = firedrake.Function(Q)
q2 = firedrake.Function(Q)

Rather than directly invoke `firedrake.solve`, we'll create some intermediate objects that will make repeated solves of the same linear system much faster.
The function `firedrake.replace` takes an argument (in this case `q`) of a form and replaces every instance of it with another function.
We're using this mechanism so that we don't have to recreate the flux objects with the intermediate stages `q1`, `q2`.

In [ ]:
problem1 = LinearVariationalProblem(m, F, δq)
problem2 = LinearVariationalProblem(m, replace(F, {q: q1}), δq)
problem3 = LinearVariationalProblem(m, replace(F, {q: q2}), δq)

Since the mass matrix for DG methods is block diagonal, the following solver parameters give us an exact solver for the mass matrix inverse.

In [ ]:
parameters = {
    'ksp_type': 'preonly',
    'pc_type': 'bjacobi',
    'sub_pc_type': 'ilu'
}

Finally we create some linear solver objects corresponding to these problems.

In [ ]:
solver1 = LinearVariationalSolver(problem1, solver_parameters=parameters)
solver2 = LinearVariationalSolver(problem2, solver_parameters=parameters)
solver3 = LinearVariationalSolver(problem3, solver_parameters=parameters)

To make things extra sharp, we can also add in a limiter.

In [ ]:
limiter = firedrake.VertexBasedLimiter(Q)

for step in range(num_steps + 1):
    solver1.solve()
    q1.assign(q + δq)
    
    solver2.solve()
    q2.assign(3/4*q + 1/4*(q1 + δq))
    
    solver3.solve()
    q.assign(1/3*q + 2/3*(q2 + δq))
    
    limiter.apply(q)
    
    print('.' if step % 50 == 0 else '', end='', flush=True)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tripcolor(q, cmap='viridis', axes=axes)
fig.show()

Once again, the symbolic language in firedrake makes it really easy to specify PDE, but it's still on you to decide how to solve it.
In this example, it's up to you to choose what time integration scheme to use and how long of a timestep.
In particular, you still need to satisfy the [Courant-Friedrichs-Lewy condition](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition).

As a sanity check, we can show that the total mass is roughly conserved.

In [ ]:
print(firedrake.assemble(q0 * dx))
print(firedrake.assemble(q * dx))

The total variation, on the other hand, appears to be increasing, which I think is bad?

...Is Randy here?

In [ ]:
tv = sqrt(inner(grad(q), grad(q))) * dx + abs(q('+') - q('-')) * dS

print(firedrake.assemble(replace(tv, {q: q0})))
print(firedrake.assemble(tv))